In [2]:
%pip install --quiet --upgrade pip 
%pip install numpy --quiet
%pip install Pandas --quiet
%pip install sklearn --quiet
%pip install ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Horse Racing Results Predictor using Linear regression #

Use simple linear regression techniques to determine if we can reliably predict the **speed** of a horse in a given race based on its previous performance. 

Data used here is derived from the features extracted by the [Feature Analysis](https://github.com/LeeSanderson/RacingData/blob/main/Data/FeatureAnalysis.ipynb) notebook.

In [3]:
import numpy as np
import pandas as pd
import math
from abc import ABC, abstractmethod
from datetime import datetime, date

In [4]:
races = pd.read_csv("Race_Features.csv")
races['Off'] =  pd.to_datetime(races['Off'], format='%Y-%m-%d %H:%M:%S')
races.columns

Index(['RaceId', 'CourseId', 'RaceType', 'Off', 'DecimalOdds',
       'OfficialRating', 'RacingPostRating', 'TopSpeedRating',
       'DistanceInMeters', 'Going', 'Surface', 'HorseId', 'HorseName',
       'JockeyId', 'JockeyName', 'Age', 'HeadGear', 'RaceCardNumber',
       'StallNumber', 'WeightInPounds', 'FinishingPosition',
       'OverallBeatenDistance', 'RaceTimeInSeconds', 'Wins',
       'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 'Going_Firm',
       'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy',
       'Going_Soft', 'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other',
       'RaceType_SteepleChase', 'Speed', 'HorseCount', 'KnownHorseAndJockey',
       'NumberOfPriorRaces', 'LastRaceGoing', 'LastRaceSurface',
       'LastRaceDistanceInMeters', 'LastRaceWeightInPounds', 'LastRaceSpeed',
       'DaysRested', 'LastRaceDecimalOdds', 'LastRaceOfficialRating',
       'LastRaceRacingPostRating', 'LastRaceTopSpeedRating',
       'LastRaceAvgRelFinishi

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

race_info = ['RaceId', 'CourseId', 'Off', 'HorseId', 'HorseName', 'HorseCount', 'JockeyId', 'JockeyName', 'DecimalOdds', 'Wins', 'FinishingPosition']
predictors = ([
    'DistanceInMeters', 
    'WeightInPounds', 
    'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 
    'Going_Firm', 'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy', 'Going_Soft', 
    'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other', 'RaceType_SteepleChase', 

    'LastRaceDistanceInMeters', 
    'LastRaceWeightInPounds', 
    'LastRaceSpeed',
    'DaysRested',
    'LastRaceAvgRelFinishingPosition', 
    'LastRaceSurface_AllWeather', 'LastRaceSurface_Dirt', 'LastRaceSurface_Turf', 
    'LastRaceGoing_Good', 'LastRaceGoing_Good_To_Soft', 'LastRaceGoing_Soft', 'LastRaceGoing_Good_To_Firm', 'LastRaceGoing_Firm', 'LastRaceGoing_Heavy', 
    'LastRaceRaceType_Other', 'LastRaceRaceType_Hurdle', 'LastRaceRaceType_SteepleChase', 'LastRaceRaceType_Flat', 
    
    'JockeyNumberOfPriorRaces',
    'DaysSinceJockeyLastRaced',     
    'JockeyWinPercentage',
    'JockeyTop3Percentage',
    'JockeyAvgRelFinishingPosition'
    ])
prediction = ["Speed"]

train = races[race_info + predictors + prediction].dropna().copy()

In [6]:
# Cap rested days
train.loc[train["DaysRested"] > 10, "DaysRested"] = 10
train.loc[train["DaysSinceJockeyLastRaced"] > 10, "DaysSinceJockeyLastRaced"] = 10

# Cap prior races
# train.loc[train["JockeyNumberOfPriorRaces"] > 400] = 400 (negative effect)

train.to_csv("Clean_Race_Features.csv", index=False)

train[predictors].describe()

,DistanceInMeters,WeightInPounds,Surface_AllWeather,Surface_Dirt,Surface_Turf,Going_Firm,Going_Good,Going_Good_To_Firm,Going_Good_To_Soft,Going_Heavy,Going_Soft,RaceType_Flat,RaceType_Hurdle,RaceType_Other,RaceType_SteepleChase,LastRaceDistanceInMeters,LastRaceWeightInPounds,LastRaceSpeed,DaysRested,LastRaceAvgRelFinishingPosition,LastRaceSurface_AllWeather,LastRaceSurface_Dirt,LastRaceSurface_Turf,LastRaceGoing_Good,LastRaceGoing_Good_To_Soft,LastRaceGoing_Soft,LastRaceGoing_Good_To_Firm,LastRaceGoing_Firm,LastRaceGoing_Heavy,LastRaceRaceType_Other,LastRaceRaceType_Hurdle,LastRaceRaceType_SteepleChase,LastRaceRaceType_Flat,JockeyNumberOfPriorRaces,DaysSinceJockeyLastRaced,JockeyWinPercentage,JockeyTop3Percentage,JockeyAvgRelFinishingPosition
count,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000,284008.000000
mean,2274.045316,153.937759,0.237293,0.018440,0.744222,0.027851,0.465973,0.094804,0.222902,0.068868,0.119391,0.007010,0.185512,0.728420,0.079058,2238.139785,153.808273,15.118013,9.764591,0.529874,0.233726,0.018021,0.748173,0.463174,0.222113,0.120796,0.097959,0.027003,0.068748,0.730677,0.184192,0.073382,0.011750,400.086980,3.038355,0.110663,0.327923,0.551156
std,1147.235990,15.911181,0.425424,0.134535,0.436298,0.164547,0.498842,0.292944,0.416194,0.253230,0.324249,0.083434,0.388713,0.444775,0.269829,1126.084070,15.838142,1.529288,0.992018,0.193289,0.423201,0.133026,0.434063,0.498643,0.415668,0.325891,0.297259,0.162092,0.253026,0.443609,0.387641,0.260763,0.107757,395.920138,2.799334,0.063913,0.124844,0.070098
min,804.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,804.000000,119.000000,9.763058,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.052632
25%,1407.000000,144.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1407.000000,144.000000,13.800707,10.000000,0.401786,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,103.000000,1.000000,0.071429,0.245098,0.511147
50%,1809.000000,149.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1709.000000,149.000000,15.553836,10.000000,0.519209,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,274.000000,2.000000,0.103659,0.324324,0.546866
75%,3217.000000,165.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3217.000000,164.000000,16.270551,10.000000,0.648889,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,572.000000,4.000000,0.142857,0.405959,0.589575
max,6937.000000,200.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6937.000000,199.000000,19.670069,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2413.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
# Now that we've dropped any rows with na values, calculate which races in which we can predict the speed for all horses.
train = train.drop("PredictableHorseCount", axis=1, errors='ignore')
groups = train.groupby(['RaceId']).apply(lambda g: pd.Series({'PredictableHorseCount': g['RaceId'].count()}, index=['PredictableHorseCount']))
train = pd.merge(train, groups, how='left', on=['RaceId'])

# 50% of races have 11 horse, 25% have 8 or less, 14% have 6 or less, 9% have 5 or less. Fewer horses should be more predictable
all_races_count = len(races["RaceId"].unique())
all_predictable = train[(train["HorseCount"] == train["PredictableHorseCount"]) & (train["HorseCount"] <= 5)]["RaceId"].unique().tolist()
all_predictable_count = len(all_predictable)
print(f"Possible predictable races = {all_predictable_count} out of {all_races_count} ({all_predictable_count * 100 / all_races_count}%)")

Possible predictable races = 3983 out of 37477 (10.627851749072764%)


In [8]:
from sklearn.model_selection import train_test_split

_, test_race_ids = train_test_split(all_predictable, test_size=0.2, random_state = 42)

test = train[train["RaceId"].isin(test_race_ids)]
train = train[train["RaceId"].isin(test_race_ids) == False]

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Create a pipeline that includes feature scaling and linear regression
model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2, interaction_only=False), LinearRegression())

In [10]:
#help(model.fit)

In [11]:
inputs = train[predictors]
targets = train[prediction[0]].values

# print(inputs.shape)
# print(targets.shape)

In [12]:
model.fit(inputs, targets)
test_inputs = test[predictors]
predictions = model.predict(test_inputs)

In [13]:
test_with_predictions = test.copy()
test_with_predictions["PredictedSpeed"] = predictions

In [14]:
test_with_predictions["PredictedRank"] = test_with_predictions.groupby("RaceId")["PredictedSpeed"].rank(method="dense", ascending=False)

In [26]:
(test_with_predictions[test_with_predictions["RaceId"] == 789029]
 [["RaceId", "HorseId", "HorseName", "FinishingPosition", "PredictedRank", "Speed", "PredictedSpeed", "Speed", "HorseCount", "DecimalOdds"]]
 #.sort_values("PredictedRank")
 .head(200))

,RaceId,HorseId,HorseName,FinishingPosition,PredictedRank,Speed,PredictedSpeed,Speed,HorseCount,DecimalOdds
1183,789029,2951763,Chalk Stream,1,2.0,15.562294,14.998642,15.562294,5,2.2
1184,789029,3142176,Barn Owl,2,3.0,15.506711,14.981018,15.506711,5,3.5
1185,789029,3481885,Carrigillihy,3,1.0,15.457025,15.017410,15.457025,5,4.5
1186,789029,3098714,Grantley,4,5.0,15.293678,14.686554,15.293678,5,15.0
1187,789029,3098690,Chase The Dollar,5,4.0,15.144306,14.841324,15.144306,5,15.0


In [16]:
correct_winners = test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] == 1)]

correct_winners_count = len(correct_winners)
incorrect_winners_count = len(test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] != 1)])

winnings = correct_winners["DecimalOdds"].sum()

print(f"Correct winners {correct_winners_count}, Incorrect winners {incorrect_winners_count}")
print(f"Win percentage {100.0 * correct_winners_count / incorrect_winners_count}")
print(f"Winnings {winnings}, losses {incorrect_winners_count}, diff  {winnings - incorrect_winners_count}")

Correct winners 258, Incorrect winners 540
Win percentage 47.77777777777778
Winnings 809.7587717837717, losses 540, diff  269.75877178377175


In [17]:
# Base = 23.3630494927759
# PolynomialFeatures(degree=2, interaction_only=True) = 25.01557632398754
# With PolynomialFeatures(degree=2, interaction_only=False) = 25.13252260679763
# With rest days capped =  25.210608424336975
# with max 11 horses in race = 28.612597776862906 (£467)
# with max 8 horses in race = 31.598513011152416 (£336)
# with max 6 horses in race = 38.113207547169814 (£343)
# with max 5 horses in race = 47.77777777777778 (£269)